# Day 23: Crab Cups

The small crab challenges you to a game! The crab is going to mix up some cups, and you have to predict where they'll end up.

The cups will be arranged in a circle and labeled clockwise (your puzzle input). For example, if your labeling were 32415, there would be five cups in the circle; going clockwise around the circle from the first cup, the cups would be labeled 3, 2, 4, 1, 5, and then back to 3 again.

Before the crab starts, it will designate the first cup in your list as the current cup. The crab is then going to do 100 moves.

Each move, the crab does the following actions:

    The crab picks up the three cups that are immediately clockwise of the current cup. They are removed from the circle; cup spacing is adjusted as necessary to maintain the circle.
    The crab selects a destination cup: the cup with a label equal to the current cup's label minus one. If this would select one of the cups that was just picked up, the crab will keep subtracting one until it finds a cup that wasn't just picked up. If at any point in this process the value goes below the lowest value on any cup's label, it wraps around to the highest value on any cup's label instead.
    The crab places the cups it just picked up so that they are immediately clockwise of the destination cup. They keep the same order as when they were picked up.
    The crab selects a new current cup: the cup which is immediately clockwise of the current cup.

For example, suppose your cup labeling were 389125467. If the crab were to do merely 10 moves, the following changes would occur:

```text
-- move 1 --
cups: (3) 8  9  1  2  5  4  6  7 
pick up: 8, 9, 1
destination: 2

-- move 2 --
cups:  3 (2) 8  9  1  5  4  6  7 
pick up: 8, 9, 1
destination: 7

-- move 3 --
cups:  3  2 (5) 4  6  7  8  9  1 
pick up: 4, 6, 7
destination: 3

-- move 4 --
cups:  7  2  5 (8) 9  1  3  4  6 
pick up: 9, 1, 3
destination: 7

-- move 5 --
cups:  3  2  5  8 (4) 6  7  9  1 
pick up: 6, 7, 9
destination: 3

-- move 6 --
cups:  9  2  5  8  4 (1) 3  6  7 
pick up: 3, 6, 7
destination: 9

-- move 7 --
cups:  7  2  5  8  4  1 (9) 3  6 
pick up: 3, 6, 7
destination: 8

-- move 8 --
cups:  8  3  6  7  4  1  9 (2) 5 
pick up: 5, 8, 3
destination: 1

-- move 9 --
cups:  7  4  1  5  8  3  9  2 (6)
pick up: 7, 4, 1
destination: 5

-- move 10 --
cups: (5) 7  4  1  8  3  9  2  6 
pick up: 7, 4, 1
destination: 3

-- final --
cups:  5 (8) 3  7  4  1  9  2  6 
```

In the above example, the cups' values are the labels as they appear moving clockwise around the circle; the current cup is marked with ( ).

After the crab is done, what order will the cups be in? Starting after the cup labeled 1, collect the other cups' labels clockwise into a single string with no extra characters; each number except 1 should appear exactly once. In the above example, after 10 moves, the cups clockwise from 1 are labeled 9, 2, 6, 5, and so on, producing 92658374. If the crab were to complete all 100 moves, the order after cup 1 would be 67384529.

Using your labeling, simulate 100 moves. What are the labels on the cups after cup 1?

In [1]:
# Python imports
from itertools import islice

import networkx as nx

from tqdm.notebook import tqdm

My first thought was to represent the whole cup game as a class, implementing the game state as a list of `int`s (cups), and shuffling list elements as if we were really playing the game.

In [2]:
class Cups:
    
    def __init__(self, cups):
        self.cups = [int(_) for _ in cups]
        
    def move(self):
        # Make pickup
        pickup = self.cups[1:4]
        cups = [self.cups[0]] + self.cups[4:]
        
        # Find index of insert point
        val = cups[0] - 1
        while True:
            if val in cups:
                idx = cups.index(val)
                break
            else:
                val -= 1
                if val < 1:
                    val = max(cups)

        # Insert pickup
        self.cups = cups[:idx+1] + pickup + cups[idx+1:]
        
        # Rotate cups
        self.cups = self.cups[1:] + [self.cups[0]]

    @property
    def labels(self):
        idx = self.cups.index(1)
        return "".join([str(_) for _ in self.cups[idx+1:] + self.cups[:idx]])
        
    def __str__(self):
        return f"{self.cups}"
        
    def __repr__(self):
        return f"<Cups {id(self)}: {self.cups}"

This works.

In [3]:
cups = Cups("389125467")
print(cups)

for _ in range(10):
    cups.move()
    print(cups)

[3, 8, 9, 1, 2, 5, 4, 6, 7]
[2, 8, 9, 1, 5, 4, 6, 7, 3]
[5, 4, 6, 7, 8, 9, 1, 3, 2]
[8, 9, 1, 3, 4, 6, 7, 2, 5]
[4, 6, 7, 9, 1, 3, 2, 5, 8]
[1, 3, 6, 7, 9, 2, 5, 8, 4]
[9, 3, 6, 7, 2, 5, 8, 4, 1]
[2, 5, 8, 3, 6, 7, 4, 1, 9]
[6, 7, 4, 1, 5, 8, 3, 9, 2]
[5, 7, 4, 1, 8, 3, 9, 2, 6]
[8, 3, 7, 4, 1, 9, 2, 6, 5]


It's fast enough for a solution to the test:

In [4]:
cups = cups = Cups("389125467")
for _ in range(100):
    cups.move()
cups.labels

'67384529'

And fast enough for the real puzzle, too:

In [5]:
cups = Cups("219748365")
for _ in range(100):
    cups.move()
cups.labels

'35827964'

## Part Two

Due to what you can only assume is a mistranslation (you're not exactly fluent in Crab), you are quite surprised when the crab starts arranging many cups in a circle on your raft - one million (1000000) in total.

Your labeling is still correct for the first few cups; after that, the remaining cups are just numbered in an increasing fashion starting from the number after the highest number in your list and proceeding one by one until one million is reached. (For example, if your labeling were 54321, the cups would be numbered 5, 4, 3, 2, 1, and then start counting up from 6 until one million is reached.) In this way, every number from one through one million is used exactly once.

After discovering where you made the mistake in translating Crab Numbers, you realize the small crab isn't going to do merely 100 moves; the crab is going to do ten million (10000000) moves!

The crab is going to hide your stars - one each - under the two cups that will end up immediately clockwise of cup 1. You can have them if you predict what the labels on those cups will be when the crab is finished.

In the above example (389125467), this would be 934001 and then 159792; multiplying these together produces 149245887792.

Determine which two cups will end up immediately clockwise of cup 1. What do you get if you multiply their labels together?

First, we try the solution above on the test puzzle:

In [6]:
cups = Cups([int(_) for _ in "389125467"] + list(range(10, 1000001)))
for _ in tqdm(range(100)):
    cups.move()

This is too slow for the full problem. We're only getting about 10 moves per second, so 10m moves would take hours.

My first thought was to use a circular graph representation, where each node contains the cup number/label, and links forward to the next node:

In [7]:
class CupsGraph:
    
    def __init__(self, cups):
        self.circle = nx.DiGraph()
        cups = [int(_) for _  in cups]
        
        # Add first node
        last_label = cups[0]
        self.circle.add_node(last_label)
        
        # Add rest of nodes, and edges
        for label in cups[1:]:
            self.circle.add_node(label)
            self.circle.add_edge(last_label, label)
            last_label = label
            
        # Close circle
        self.circle.add_edge(last_label, cups[0])
        
        # Set current node
        self.curnode = int(cups[0])
            
        
    def move(self):
        # Get region to move
        curnode, pickup_start, pickup, pickup_end, next_node = \
            tuple(islice(nx.traversal.bfs_tree(self.circle, self.curnode), 5))

        # Link current node to the next node but three
        self.circle.remove_edge(curnode, pickup_start)
        self.circle.remove_edge(pickup_end, next_node)
        self.circle.add_edge(curnode, next_node)
        
        # Identify node for returning pickup
        retval = self.curnode - 1
        while True:
            if retval not in (0, pickup_start, pickup, pickup_end):
                break
            else:
                retval -= 1
                if retval < 1:
                    retval = max(self.circle.nodes)
                    
        # Return pickup
        ins_left, ins_right = tuple(islice(nx.traversal.bfs_tree(self.circle, retval), 2))
        self.circle.remove_edge(ins_left, ins_right)
        self.circle.add_edge(ins_left, pickup_start)
        self.circle.add_edge(pickup_end, ins_right)
        
        # Move to next node
        self.curnode = next_node

    @property
    def labels(self):
        return "".join([str(_) for _ in nx.traversal.bfs_tree(self.circle, 1)])[1:]
        
    def __str__(self):
        return f"{self.circle.nodes()}"
        
    def __repr__(self):
        return f"<Cups {id(self)}>: Size: {len(self.circle)}"

In [8]:
cups = CupsGraph([int(_) for _ in "389125467"] + list(range(10, 1000001)))

# for _ in tqdm(range(10)):
#     cups.move()

But this is even slower than the first solution! I'm sure that this is a feasible approach, but the `networkx` overhead is possibly the issue.

We can strip down the graph representation to a `dict` of `Cup` node objects. We implement each `Cup` as an object with `value` (label) and `nextcup` (implied edge to next `Cup`) attributes. Storing these in a dictionary makes finding individual cups fast, and we only need to manipulate edges of the pickup set and its neighbours.

Rather than define a set of cups as a class and call a `.move()` method, this time we have a `play()` function that plays the game for a certain number of rounds, and gives us the first ten labels following from label `1`.

In [9]:
class Cup:
    
    def __init__(self, value, nextcup):
        self.value = value
        self.nextcup = nextcup
        
    def __repr__(self):
        return f"<Cup {id(self)}>: {self.value} -> {self.nextcup}"
        
def play(cups, rounds):
    cupints = [int(_) for _ in cups]
    curcup = cupints[0]
    
    cupdict = {}
    
    for nextcup in cupints[1:]:
        cupdict[curcup] = Cup(curcup, nextcup)
        curcup = nextcup
    cupdict[nextcup] = Cup(nextcup, cupints[0])
    
    curcup = cupdict[cupints[0]]
    
    for round in tqdm(range(rounds)):
        # Identify pickups
        pstart = cupdict[curcup.nextcup]
        pmid = cupdict[pstart.nextcup]
        pend = cupdict[pmid.nextcup]
        nextcup = cupdict[pend.nextcup]
        
        # Relink original cups
        curcup.nextcup = nextcup.value

        # Identify node for returning pickup
        retval = curcup.value - 1
        while True:
            if retval not in (0, pstart.value, pmid.value, pend.value):
                break
            else:
                retval -= 1
                if retval < 1:
                    retval = max(cupdict)
        
        # Relink pickups
        cupdict[pend.value].nextcup = cupdict[retval].nextcup
        cupdict[retval].nextcup = pstart.value
        
        # Move forward
        curcup = nextcup
        
    # Print list of cups, starting at 1
    curcup = 1
    cupvals = []
    for _ in range(10):
        cupvals.append(curcup)
        curcup = cupdict[curcup].nextcup
        
    # Return first ten labels after label 1
    return (cupvals[1:])

In [10]:
cups = "389125467"
play(cups, 10)

[9, 2, 6, 5, 8, 3, 7, 4, 1]

In [11]:
cups = [int(_) for _ in "389125467"] + list(range(10, 1000001))
result = play(cups, 10000000)
result[0] * result[1]

149245887792

In [12]:
cups = [int(_) for _ in "219748365"] + list(range(10, 1000001))
result = play(cups, 10000000)
result[0] * result[1]

5403610688